In [11]:
import math
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [12]:
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = (math.sin(dlat / 2) ** 2 +
    math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2)
    c = 2* math.asin(math.sqrt(a))
    return R * c

In [13]:
denver_airport = ("Denver Airport", (39.8571391027151, -104.67682422525337))

resorts = [
    denver_airport,
    ("Vail", (39.60634275473704, -106.35502534758554)),
    ("Beaver Creek", (39.601840601122596, -106.53163266107806)),
    ("Breckenridge", (39.48111351412133, -106.07307234912773)),
    ("Keystone", (39.58186424927065, -105.94362588806374)),
    ("A-Basin", (39.634289749362566, -105.8714993899124)),
    ("Eldora", (39.937376571357305, -105.58271078989766)),
    ("Canyons", (40.685834569379516, -111.55630231684529)),
    ("Heavenly", (38.9288904430737, -119.90519040528994)),
    ("Northstar", (39.26481569676928, -120.13316534575092)),
    ("Kirkwood", (38.68503611762454, -120.06520941879404)),
    ("Afton Alps", (44.85782048122588, -92.78779547430095)),
    ("Mt. Brighton", (42.5410196899345, -83.81160753394613)),
    ("Verbier", (46.09708737973264, 7.227272162830475)),
    ("Arlberg", (47.129520437009155, 10.263859008628906)),
    ("The 3 Valleys", (45.34154189826385, 6.586552723873957))
]

In [14]:
drive_cost_per_mile = 0.58
flight_cost_per_mile = 0.14
threshold_miles = 400
lodging_per_stop = 120

def km_to_miles(km):
    return km / 1.60934

In [15]:
num_locations = len(resorts)
cost_matrix = []
for i in range(num_locations):
    row = []
    for j in range(num_locations):
        if i == j:
            row.append(0)
        else:
            dist_km = haversine(resorts[i][1], resorts[j][1])
            dist_miles = km_to_miles(dist_km)

            if dist_miles > threshold_miles:
                transport_cost = dist_miles * flight_cost_per_mile
            else:
                transport_cost = dist_miles * drive_cost_per_mile

            lodging_cost = lodging_per_stop if j != 0 else 0

            total_cost = transport_cost = transport_cost + lodging_cost

            row.append(int(round(total_cost)))
    cost_matrix.append(row)

In [16]:
manager = pywrapcp.RoutingIndexManager(len(cost_matrix), 1, 0)
routing = pywrapcp.RoutingModel(manager)

In [17]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return cost_matrix[from_node][to_node]

In [18]:
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [19]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

In [20]:
solution = routing.SolveWithParameters(search_parameters)

In [21]:
if solution:
    index = routing.Start(0)
    route = []
    total_cost = 0
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(resorts[node][0])
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        arc_cost = routing.GetArcCostForVehicle(previous_index, index, 0)
        total_cost += arc_cost
    route.append(resorts[manager.IndexToNode(index)][0])

    print("Optimal Route (by location names):")
    print("->".join(route))
    print(f"Total route cost:, ${total_cost:,}")
else:
        print("No Solution found!")

Optimal Route (by location names):
Denver Airport->Eldora->A-Basin->Keystone->Breckenridge->Vail->Beaver Creek->Kirkwood->Heavenly->Northstar->Canyons->Arlberg->Verbier->The 3 Valleys->Mt. Brighton->Afton Alps->Denver Airport
Total route cost:, $3,704
